In [1]:
import requests as r
from bs4 import BeautifulSoup
import re
from time import gmtime, strftime
import multiprocessing
import time
from multiprocessing import Pool, Lock

In [2]:
url='https://tw.mall.yahoo.com/search?m=list&sid=daiso&s=-sc_salerank&view=both&b=0'
res = r.get(url)
soup = BeautifulSoup(res.text,'lxml')

In [3]:
def daiso(link):
    res = r.get(link)
    time.sleep(0.01)
    soup = BeautifulSoup(res.text,'lxml')
    name = soup.select_one('div > h1 > span') #商品名稱
    price = soup.select_one('div > span.price')
    current_time = strftime("%Y-%m-%d %H:%M:%S", gmtime()) #爬下商品時間
    store = '大創'
    fee = 50 #運費
    a = soup.select('div.left > ul > li') #銷售數量
    sell = a[3].text
    data= '{},{},{},{},{},{},{}\n'.format(store,current_time,name.text,price.text,link,fee,sell)
    data.encode('utf-8')
    product = data
    #     print(products)
    print('.', end="")
    #with lock:
    with open ('daiso.csv','a') as fw:
        fw.write(product)
        fw.close()

        

In [4]:
#一頁有48個產品,第一頁網址：https://tw.mall.yahoo.com/search?m=list&sid=daiso&s=-sc_salerank&view=both&b=48
#             第二頁網址：https://tw.mall.yahoo.com/search?m=list&sid=daiso&s=-sc_salerank&view=both&b=96
all_links=[]
for i in range(0,125):#124頁
    page = i*48
    res = r.get('https://tw.mall.yahoo.com/search?m=list&sid=daiso&s=-sc_salerank&view=both&b={}'.format(page))
    soup = BeautifulSoup(res.text,'lxml')
    links = soup.select('div > table > tbody > tr > td > a') # 一頁有48個網址
    link_list = [link['href'] for link in links] # 把48個網址丟入link_list中
    all_links += link_list # 把所有網址丟入all_links中(每一頁的所有網址 48*頁數 個網址)
    print('=', end="")


pool = multiprocessing.Pool(processes=2)
lock = multiprocessing.Manager().Lock()
for link in all_links:
#     print(link) 
#     daiso(lock,link)
    pool.apply_async(daiso,(link,))
    time.sleep(0.01)
    #pool.apply_async(<def>, (lock,<link>, ))
    
pool.close()
pool.join()




=============================================================================================================================...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................